# JAY BAKSHI : UB ID - 50206954

Installing and importing relevant packages and libraries.

In [ ]:
install.packages('dpylr', repos = 'https://cran.r-project.org')

In [ ]:
install.packages('DBI', repos="https://cran.r-project.org")

In [ ]:
install.packages('RSQLite', repos = 'http://cran.r-project.org')

In [ ]:
install.packages('sqldf', repos="https://cran.r-project.org")

In [ ]:
library(dplyr)

In [ ]:
library(DBI)

In [ ]:
library(RSQLite)

In [ ]:
library(sqldf)

### Downloaded the dataset at https://www.kaggle.com/hugomathien/soccer

Accessing the database.sqlite using DBI methods -

In [ ]:
dbconn <- dbConnect(SQLite(), dbname='database.sqlite')

In [ ]:
class(dbconn)

In [ ]:
dbListTables(dbconn)

Accessing the database.sqlite using dplyr methods -

In [ ]:
my_db <- src_sqlite("database.sqlite", create=T)

In [ ]:
class(my_db)

In [ ]:
src_tbls(my_db)

<b>Extracting the relevant columns</b>
<br/>Match - date, match_api_id, home_team_api_id, away_team_api_id, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, home_team_goal, away_team_goal, foulcommit, card
<br/>Player - player_api_id, player_name
<br/>Player_Attributes - player_api_id, date, overall_rating, gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes
<br/>Team - team_api_id, team_long_name
<br/>Team_Attributes - team_api_id, buildUpPlaySpeed, buildUpPlayDribbling, buildUpPlayPassing, chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, defencePressure, defenceAggression, defenceTeamWidth

<code>playerrel</code> stores all the player and player_attributes relevant columns

In [ ]:
playerrel.df <- data.frame(tbl(my_db, sql("SELECT DISTINCT Player.player_api_id, player_name, 
                                            date, overall_rating, 
                                            (gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes) AS gk_all
                                            FROM Player, Player_Attributes
                                            WHERE Player.player_api_id = Player_Attributes.player_api_id")))

In [ ]:
head(playerrel.df, 10)

Checking up on the dimension and class of the playerrel data frame

In [ ]:
class(playerrel.df)
dim(playerrel.df)

In [ ]:
tbl(my_db, sql("SELECT COUNT(*) 
                FROM Player, Player_Attributes
                WHERE Player.player_api_id = Player_Attributes.player_api_id"))

Writing this to the <strong>player.csv</strong> file and saving it locally.

In [ ]:
write.csv(playerrel.df, file="player.csv")

<code>matchrel</code> will store the relevant columns of match table, this I'll combine with the relevant columns from team and team_Attributes below to form <strong>team.csv</strong>

In [ ]:
matchrel <- data.frame(tbl(my_db, sql("SELECT DISTINCT match_api_id, home_team_api_id, league_id,
                                            away_team_api_id, home_player_1, home_player_2, 
                                            home_player_3, home_player_4, home_player_5, home_player_6,
                                            home_player_7, home_player_8, home_player_9, home_player_10,
                                            home_player_11, away_player_1, away_player_2, away_player_3, 
                                            
                                            away_player_4, away_player_5, away_player_6, away_player_7, 
                                            away_player_8, away_player_9, away_player_10, away_player_11, 
                                            home_team_goal, away_team_goal
                                            
                                            FROM Match")))

Testing and making sure of the data in matchrel

In [ ]:
head(matchrel, 10)

In [ ]:
class(matchrel)
dim(matchrel)

<code>teamrel</code> is storing all the relevant data and columns from team and team_attributes table.

In [ ]:
teamrel <- data.frame(tbl(my_db, sql("SELECT DISTINCT Team.team_api_id, team_long_name,
                                        (buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing) AS buildUpPlay,
                                        (chanceCreationPassing + chanceCreationCrossing + chanceCreationShooting) AS chanceCreation, 
                                        (defencePressure + defenceAggression + defenceTeamWidth) AS defence
                                        FROM Team, Team_Attributes
                                        WHERE Team.team_api_id = Team_Attributes.team_api_id")))

Keeping a check on the data in teamrel data frame

In [ ]:
head(teamrel, 10)

In [ ]:
class(teamrel)
dim(teamrel)

<code>teammatchrel</code> is going to store the combination of the columns across match, team and team_attributes to later on write this down in <strong>team.csv</strong>

In [ ]:
teammatchrel <- sqldf("SELECT DISTINCT *
                         FROM (SELECT DISTINCT *
                             FROM matchrel
                             LEFT JOIN teamrel
                             ON matchrel.home_team_api_id = teamrel.team_api_id) AS Temp
                                            LEFT JOIN teamrel
                                            ON Temp.away_team_api_id = teamrel.team_api_id")

In [ ]:
colnames(teammatchrel)

In [ ]:
head(teammatchrel, 10)

In [ ]:
dim(teammatchrel)
str(teammatchrel)

In [ ]:
write.csv(teammatchrel, file="team.csv")

With this Lab 2 Activity 2 is finished.